Encontrei o tutorial em https://medium.com/ensina-ai/prevendo-roubos-de-celulares-em-s%C3%A3o-paulo-4ee5b0845fc2 
Achei muito interessante aprendi várias tecnicas como:



Caso esteja vendo no github utilize o __NBViewer__ para ver a interação dos gráficos
Link :